In [14]:
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [5]:
img_channels = 3
img_rows = 32
img_cols = 32

In [6]:
batch_size = 128
epochs = 40
number_classes = 10
verbose = 1
validation_split = 0.2
optimizer = RMSprop()

In [10]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train.shape, X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [11]:
y_train = to_categorical(y_train, number_classes)
y_test = to_categorical(y_test, number_classes)

In [12]:
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

In [18]:
model = Sequential()
model.add(Conv2D(32, kernel_size=3,
                 padding="same",
                 input_shape=(img_rows, img_cols, img_channels),
                 activation="relu"))
model.add(Conv2D(32, kernel_size=3, padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, padding="same",activation="relu"))
model.add(Conv2D(64, kernel_size=3, padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(number_classes,activation="relu"))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 8, 8, 64)         

In [19]:
model.compile(loss="categorical_crossentropy",
              optimizer = optimizer,
              metrics = ["accuracy"])

In [22]:
datagen = ImageDataGenerator(featurewise_center=False, #set input mean to 0 over the dataset
                             samplewise_center=False, # set each sample mean to 0
                             featurewise_std_normalization=False, # divide inputs by std of the dataset,
                             samplewise_std_normalization=False,  # divide each input by its std
                             zca_whitening=False,  # apply ZCA whitening,
                             rotation_range=0, # randomly rotate images in the range
                             width_shift_range=0.1, # randomly shift images horizontally
                             height_shift_range=0.1,# randomly shift images vertically
                             horizontal_flip=True, # randomly flip images
                             vertical_flip=False # randomly flip images
                            )

datagen.fit(X_train)

In [28]:
history = model.fit(X_train,y_train, batch_size=256,
                    epochs=2, validation_split=validation_split, verbose=verbose)

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
12800/40000 [========>.....................] - ETA: 1:32 - loss: 2.2874 - accuracy: 0.2153

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(X_test,y_test, batch_size=batch_size, verbose=verbose)
print("\n Test score:", score[0])
print("Test accuracy:", score[1])

In [ ]:
model_json = model.to_json()
open("cifar10_arc.json","w").write(model_json)
model.save_weights("cifar10_weights.h5", overwrite=True)

In [ ]:
print(history.history.keys())
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()